`TODO`:
- [ ] revise outline below to match flow of `expt1_dev.py`
- [ ] abstract over params in `expt1_dev.py`
- [ ] once finished w `expt1_dev.py`, move it to this nb
- [ ] figger out clean way to do imports for class module (tricky) 
- [ ] decide whether to use wrapper classes at all
- [ ] determine if TypeC even needs a class (def needs minimal text prep)
- [ ] add sub-expt w train on ~70% and eval on length subsets 
- [ ] write + test out crossval func that can apply to any of the clf types


> **Note**: old version of this nb w outdated outline etc. now lives in `'/ignore/scrapped-but-useful'` (new version is `'/experiments/expt1/expt1.ipynb'`)

<hr><hr>
<br><br>

## Experiment 1
#### Document length and classification accuracy
<hr>

###### Part 1: Define experiment 
1. define space of classifiers (and hypers) to consider 
2. import classes for managing arbitrary `sklearn` and `keras` models 
3. write model evaluation function 
4. write performance curve flattener 

###### Part 2: Prepare data 
1. import text preprocessing functions/classes 
2. load and preprocess text (need >1 format for diff clf styles)
3. split data into length subsets 
4. set aside 30% of data for evaluation, stratified by length and label

###### Part 3: Conduct experiment 
1. for each train subset, get 5-fold crossval F1 for each clf
2. for each fit, generate and save preds on evaluation set 

###### Part 4: Evaluate results
1. plot the crossval F1 scores across subsets and clfs
2. plot performance on validation set for each clf and subset 


<br>
#### Part 1: Define experiment
<hr>

##### 1.1 Define space of classifiers 


##### 1.2 Import what's needed for each model type

##### 1.3 Import what's needed for preprocssing text

##### 1.4 Import what's needed for evaluation

##### 1.5 Import what's needed for postprocessing results

<br>
#### Part 2: Prepare data
<hr>

<br>
#### Part 3: Conduct experiment
<hr>

<br>
#### Part 4: Evaluate results
<hr>

<br><br><br><br><br>
<hr><hr>
### sqrache areyaya